""""

1. Tried pulling directly from the website source api, got my ass kicked
2. Skywalk API is working and they fixed some requests
3. Testing dashboard combined with Apartments.com data-puller

""""

In [5]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
from prophet import Prophet

import censusdata
import us
import pandas as pd


import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
import requests
from bs4 import BeautifulSoup
import json

appfolio_api_key = "wc9DeJ9UHoJKkEsgr9CN4W.8AnBp5DgrPE2HNdNMQ1HxwDlBkJnYRZDyGvletEznkE"
payload = {"X-API-KEY" : appfolio_api_key}


In [4]:
from datetime import date

class Appfolio_Report():
    def __init__(self):
        self.today = date.today()
    
    def initialize_spreadsheet(self, spreadsheet_name):
        # use creds to create a client to interact with the Google Drive API
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(creds)

        # Find a workbook by name and open the first sheet
        # Make sure you use the right name here.
        if "docs.google.com" in spreadsheet_name:
            spreadsheet = client.open_by_url(spreadsheet_name)
        else:
            spreadsheet = client.open(spreadsheet_name)
        self.spreadsheet =  spreadsheet

    def connect_worksheet(self):
        """connect to spreadsheet"""
        today=self.today
        #initialize new spreadsheet for each day it is updated
        try:
            sheet = self.spreadsheet.add_worksheet(title=str(today), rows=100, cols=20)
        except gspread.exceptions.APIError:
            sheet = self.spreadsheet.worksheet(str(today))

        #Create Date Updated        
        sheet.batch_update([
            {
                "range" : 'A1:A2', 
                "values" : [["Date Updated"], [str(today)]]
            }
        ])
        self.sheet = sheet
    
    def populate_property_data(self):
        """start pulling information from skywalk appfolio api"""    
        
        sheet = self.sheet
        
        #Pull different properties
        property_request = requests.get('https://api.skywalkapi.com/v1/properties', headers=payload)
        print(property_request.raise_for_status())
        properties = json.loads(property_request.content)
        self.properties = properties
        
        #Loop through apartments
        for apartment in properties["data"]:
            
            #get parameters for specific properties
            self.params = { "search": "?propertyId=" + apartment["appfolioId"],}
            
            #Vacancy Rates
            unit_vacancies_request = requests.get('https://api.skywalkapi.com/v1/unit-vacancies', headers=payload, params = self.params)
            print(unit_vacancies_request.raise_for_status())
            unit_vacancies = json.loads(unit_vacancies_request.content)

            occupancy_rate = round(1 - unit_vacancies["meta"]["count"]/apartment["unitCount"], 3)
            sheet.batch_update([{
                "range" : 'A3:A5', 
                "values" : [[apartment["name"]], ["Occupancy Rate"], [occupancy_rate]]
            }])

            #update unit vacancies     
            unit_vancancy_row_header = ["name", "type", "advertisedRent", "daysVacant", "lastMoveIn", "postedToInternet", "postedToWebsite"]
            sheet.batch_update([{
                "range":"A7:G7", 
                "values" : [unit_vancancy_row_header]
            }])

            unit_row = 8 #STARTS AT 7
            for unit in unit_vacancies["data"]:
                data_to_append = []
                
                for count, col in enumerate(unit_vancancy_row_header):
                    data_to_append.append(unit[col])
                    
                range_rows = "A"+str(unit_row)+":G"+str(unit_row)
                sheet.batch_update([{
                    "range": range_rows,
                    "values": [data_to_append]
                }])

                unit_row = unit_row + 1



report = Appfolio_Report()
report.initialize_spreadsheet("https://docs.google.com/spreadsheets/d/14Loy0p4sV6xMkh1UqFWCqo8296h-4IwzvYi1_sLiGdM/edit#gid=0")
report.connect_worksheet()
report.populate_property_data()

None
None


In [219]:
balance_sheet_request = requests.get('https://api.skywalkapi.com/v1/balance-sheet', headers=payload, params = cf_params)
print(balance_sheet_request.raise_for_status())
balance_sheet = json.loads(balance_sheet_request.content)
balance_sheet

None


{'meta': {'status': 'ok',
  'path': '/v1/balance-sheet',
  'lastUpdated': 1648244034,
  'jobId': 'cd200b3f-587d-4cff-9a05-57b5bf54e3ae',
  'count': 55,
  'query': {'accounting': 'cash', 'date': '03/25/2022'}},
 'links': {},
 'data': [{'id': '0',
   'title': 'ASSETS',
   'glAccountId': None,
   'glAccountName': 'ASSETS',
   'glAccountCode': None,
   'balance': '193,811.91',
   'childIds': ['1',
    '6',
    '7',
    '8',
    '9',
    '10',
    '14',
    '18',
    '19',
    '20',
    '21',
    '22',
    '23',
    '24',
    '25',
    '26',
    '27',
    '28',
    '29',
    '30',
    '31',
    '32']},
  {'id': '1',
   'title': 'Cash',
   'glAccountId': None,
   'glAccountName': 'Cash',
   'glAccountCode': None,
   'balance': '177,226.72',
   'childIds': ['2', '3', '4'],
   'parentId': '0'},
  {'id': '2',
   'title': 'Operating Cash',
   'glAccountId': '2',
   'glAccountName': 'Operating Cash',
   'glAccountCode': '1150',
   'balance': '174,426.72',
   'childIds': [],
   'parentId': '1'},
 

In [224]:
cf_params = { "search": "?propertyId=51&accounting=cash&dateRange=this_month",}

cash_flow_request = requests.get('https://api.skywalkapi.com/v1/cash-flow', headers=payload, params = cf_params)
print(cash_flow_request.raise_for_status())
cashflow = json.loads(cash_flow_request.content)
cashflow

None


{'meta': {'status': 'ok',
  'path': '/v1/cash-flow',
  'lastUpdated': 1648243192,
  'jobId': 'a061d844-a9c3-4de8-92d1-ec62f3bb9fdf',
  'count': 244,
  'query': {'accounting': 'cash',
   'ownerPercentBased': 'No',
   'dateRange': 'this_year_to_date',
   'dateStart': '01/01/2022',
   'dateEnd': '03/25/2022'}},
 'links': {'next': '/v1/cash-flow?accounting=cash&ownerPercentBased=No&dateRange=this_year_to_date&dateStart=01%2F01%2F2022&dateEnd=03%2F25%2F2022&cursor=eyJqb2JJZCI6ImEwNjFkODQ0LWE5YzMtNGRlOC05MmQxLWVjNjJmM2JiOWZkZiIsInBhZ2UiOjJ9'},
 'data': [{'id': '1',
   'title': 'Operating Income',
   'glAccountId': None,
   'glAccountName': 'Income',
   'glAccountCode': None,
   'selectedPeriod': '284,135.48',
   'selectedPeriodAsPercent': '100.00',
   'fiscalYearToDate': '284,135.48',
   'fiscalYearToDateAsPercent': '100.00',
   'childIds': ['2', '3', '8', '9', '10', '11', '12', '51', '52']},
  {'id': '2',
   'title': 'Appfolio Mtm Additional Fee',
   'glAccountId': '112',
   'glAccountName'

# Dashboard

In [111]:
#zillow rent data
rent_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
rent_df.drop(columns= ["RegionID", "SizeRank"], inplace=True)
rent_df.head()

,RegionName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02
0,United States,1267.0,1273,1278.0,1283,1288,1294,1299,1304,1309,...,1694.0,1716,1737,1759,1781.0,1803.0,1825,1848,1870,1893
1,"New York, NY",2257.0,2267,2277.0,2287,2297,2307,2317,2327,2337,...,2553.0,2586,2619,2654,2689.0,2724.0,2760,2796,2832,2870
2,"Los Angeles-Long Beach-Anaheim, CA",1778.0,1789,1800.0,1810,1821,1832,1842,1853,1864,...,2499.0,2527,2554,2583,2611.0,2639.0,2668,2698,2727,2757
3,"Chicago, IL",1412.0,1415,1419.0,1423,1426,1430,1434,1437,1441,...,1660.0,1674,1688,1703,1717.0,1731.0,1746,1761,1776,1791
4,"Dallas-Fort Worth, TX",1116.0,1119,1123.0,1126,1130,1134,1137,1141,1145,...,1537.0,1559,1581,1603,1626.0,1648.0,1671,1694,1716,1739


In [112]:
# restacking frame
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")

# housing from ZRent
housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1645574925")
housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)

# restacking frame
reframed_housing_df = pd.DataFrame(housing_df.set_index("RegionName").stack()).reset_index()
reframed_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_housing_df.RegionName = reframed_housing_df.RegionName.str.replace(",", "")
reframed_housing_df.Date = reframed_housing_df.Date.str[:-3]

# combining rent and housing
reframe_df = pd.merge(reframed_rent_df, reframed_housing_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])
reindexed_reframed_df = reframe_df.set_index(["RegionName", "Date"])

# rent buy ratio
reindexed_reframed_df["rent_buy_ratio"] = reindexed_reframed_df["ZRent"]/reindexed_reframed_df["ZEstimate"]

# adding percentage change
reindexed_reframed_df["rent_pct_change"] = reindexed_reframed_df["ZRent"].pct_change()
reindexed_reframed_df["home_value_pct_change"] = reindexed_reframed_df["ZEstimate"].pct_change()
reindexed_reframed_df["rent_buy_pct_change"] = reindexed_reframed_df["rent_buy_ratio"].pct_change()

# removing wrong months
reindexed_reframed_df.reset_index(inplace = True)
reindexed_reframed_df = reindexed_reframed_df[reindexed_reframed_df.Date != "2014-01"]

reindexed_reframed_df.head()

,RegionName,Date,ZRent,ZEstimate,rent_buy_ratio,rent_pct_change,home_value_pct_change,rent_buy_pct_change
1,United States,2014-02,1273.0,184859.0,0.006886,0.004736,0.005740,-0.000998
2,United States,2014-03,1278.0,185731.0,0.006881,0.003928,0.004717,-0.000786
3,United States,2014-04,1283.0,186447.0,0.006881,0.003912,0.003855,0.000057
4,United States,2014-05,1288.0,187099.0,0.006884,0.003897,0.003497,0.000399
5,United States,2014-06,1294.0,187769.0,0.006891,0.004658,0.003581,0.001074


## Census Data

In [287]:
census_data = pd.read_csv("data/fully_interpolated_census_data_msa.csv")
census_data.rename(columns = {"year" : "Date"}, inplace=True)
census_data["population_change"] = census_data["population"].pct_change()
census_data["median_income_change"] = census_data["median_income"].pct_change()


### True Value

In [270]:
true_housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/mlp/Metro_mlp_uc_sfrcondo_sm_month.csv?t=1646178611")
true_housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)
reframed_true_housing_df = pd.DataFrame(true_housing_df.set_index("RegionName").stack()).reset_index()

reframed_true_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_true_housing_df.RegionName = reframed_true_housing_df.RegionName.str.replace(",", "")
reframed_true_housing_df.Date = reframed_true_housing_df.Date.str[:-3]

reframed_true_housing_df.head()

,RegionName,Date,ZEstimate
0,United States,2018-01,271600.0
1,United States,2018-02,272967.0
2,United States,2018-03,279333.0
3,United States,2018-04,288667.0
4,United States,2018-05,295633.0


# Plotting

In [273]:
def graph_layout(id_name, title):
    return html.Div(children=[
        
                html.Br(),
                html.Label(title),
                dcc.Graph(
                    id=id_name,
                )],
            style=style, draggable=True)

def create_sub_plot(filtered_df, dependent_variable):
    fig = px.line(filtered_df, x="Date", y=dependent_variable, color="RegionName", hover_name="RegionName")
    fig.update_layout(transition_duration=500, template = template)
    return fig

In [298]:
app = JupyterDash(__name__, 
            external_stylesheets=[dbc.themes.BOOTSTRAP],
            meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"}
    ],)
del app.config._read_only["requests_pathname_prefix"]
server = app.server


template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}
 

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reindexed_reframed_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    graph_layout('Zrent', 'Graph of ZRents'), 
    graph_layout('Zrent_pct_change', 'Graph of ZRents Change'), 
    
    graph_layout('ZEst', 'Graph of ZEstimates'), 
    graph_layout('ZEst_pct_change', 'Graph of ZEstimates Change'),
    
    graph_layout('Zrent_buy', 'Graph of ZRent/ZBuy'), 
    graph_layout('Zrent_buy_pct_change', 'Graph of ZRent/ZBuy Change'),
    
    graph_layout(id_name = 'median_income', title = 'Incomes (Projected 2020-2022)'), 
    graph_layout(id_name = 'median_income_change', title = 'Income Growth (Projected 2020-2022)'), 

    graph_layout(id_name = 'population', title = 'Population (Projected 2020-2022)'),
    graph_layout(id_name = 'population_change', title = 'Population Growth (Projected 2020-2022)'),
    
    #dash_table.DataTable(df.to_dict('records'),[{"name": i, "id": i} for i in df.columns], id='tbl'),
    
    ])

@app.callback(
    Output('Zrent', 'figure'),
    Output('Zrent_pct_change', 'figure'),
    Output('ZEst', 'figure'),
    Output('ZEst_pct_change', 'figure'),
    Output('Zrent_buy', 'figure'),
    Output('Zrent_buy_pct_change', 'figure'),
    Output('median_income', 'figure'),
    Output('median_income_change', 'figure'),
    Output('population', 'figure'),
    Output('population_change', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reindexed_reframed_df[reindexed_reframed_df.RegionName.isin(selected_cities)]  
    census_filtered_df = census_data[census_data.RegionName.isin(selected_cities)]  

    fig1 = create_sub_plot(filtered_df, "ZRent")
    fig2 = create_sub_plot(filtered_df, "rent_pct_change")
    
    fig3 = create_sub_plot(filtered_df, "ZEstimate")
    fig4 = create_sub_plot(filtered_df, "home_value_pct_change")
    
    fig5 = create_sub_plot(filtered_df, "rent_buy_ratio")
    fig6 = create_sub_plot(filtered_df, "rent_buy_pct_change")
    
    fig7 = create_sub_plot(census_filtered_df, "median_income")
    fig8 = create_sub_plot(census_filtered_df[census_filtered_df["Date"] != 2015], "median_income_change")
    
    fig9 = create_sub_plot(census_filtered_df[census_filtered_df["RegionName"] != "United States"], "population")
    
    fig10 = create_sub_plot(census_filtered_df[census_filtered_df["Date"] != 2015], "population_change")
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10
    
if __name__ == '__main__':
    app.run_server(debug=False)

127.0.0.1 - - [29/Mar/2022 19:32:32] "GET /_shutdown_91762656-42e1-4a1c-b685-b4bd49d69046 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2022 19:32:33] "GET /_alive_91762656-42e1-4a1c-b685-b4bd49d69046 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [29/Mar/2022 19:32:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Mar/2022 19:32:34] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


selected_cities ['United States', 'Jacksonville FL']


127.0.0.1 - - [29/Mar/2022 19:32:34] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'Dallas-Fort Worth TX']


127.0.0.1 - - [29/Mar/2022 19:32:48] "POST /_dash-update-component HTTP/1.1" 200 -


In [284]:
census_filtered_df = census_data[census_data.RegionName.isin(["United States"])]  


census_filtered_df[census_filtered_df["Date"] != 2015]

,Date,RegionName,population,median_value,gross_rent_total,median_income,median_income_change
7378,2016,United States,3.187392e+08,184925.714286,951.457143,55530.114286,0.032397
7379,2017,United States,3.208849e+08,193611.428571,982.514286,57636.228571,0.037927
7380,2018,United States,3.228813e+08,204525.714286,1019.857143,60105.914286,0.042850
7381,2019,United States,3.247286e+08,217668.571429,1063.485714,62939.171429,0.047138
7382,2020,United States,3.264266e+08,233040.000000,1113.400000,66136.000000,0.050792
7383,2021,United States,3.279755e+08,250640.000000,1169.600000,69696.400000,0.053835
7384,2022,United States,3.293751e+08,270468.571429,1232.085714,73620.371429,0.056301


# BLS

In [214]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
series_id = 'CUUR0000SA0'
data = json.dumps({"seriesid": [series_id],"startyear":"2011", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
series = json_data['Results']['series'][0]

pd.DataFrame(series["data"])

,year,period,periodName,value,footnotes
0,2020,M12,December,260.474,[{}]
1,2020,M11,November,260.229,[{}]
2,2020,M10,October,260.388,[{}]
3,2020,M09,September,260.280,[{}]
4,2020,M08,August,259.918,[{}]
...,...,...,...,...,...
115,2011,M05,May,225.964,[{}]
116,2011,M04,April,224.906,[{}]
117,2011,M03,March,223.467,[{}]
118,2011,M02,February,221.309,[{}]


# Plotting

In [57]:
### PLOTTING ###
app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]
server = app.server

template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}



app.layout = html.Div([
    html.Div(children=[
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = new_aggregate_census_df.RegionName.unique(),
                     value = ["Orlando-Kissimmee-Sanford FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    graph_layout('population_in_occupied_units', 'Graph of Population'), 
    #graph_layout('population_in_occupied_units', 'Graph of Population'), 

    graph_layout('median_value', 'Graph of Median Values'), 
    #graph_layout('median_value', 'Graph of Median Values'), 

    graph_layout('gross_rent_total', 'Graph of Gross Rents')

    ])

@app.callback(
    Output('population_in_occupied_units', 'figure'),
    Output('median_value', 'figure'),
    Output('gross_rent_total', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = new_aggregate_census_df[new_aggregate_census_df.RegionName.isin(selected_cities)]   
    
    fig1 = create_sub_plot(filtered_df, "population_in_occupied_units")
    fig2 = create_sub_plot(filtered_df, "median_value")
    fig3 =create_sub_plot(filtered_df, "gross_rent_total")
    
    return fig1, fig2, fig3
    
if __name__ == '__main__':
    app.run_server(debug=False)

/Users/nelsonlin/Desktop/AYPOClicker/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [22/Mar/2022 14:36:53] "GET /_shutdown_d881a351-e7fa-40f2-acba-efe7f8383100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2022 14:36:53] "GET /_shutdown_d881a351-e7fa-40f2-acba-efe7f8383100 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2022 14:36:53] "GET /_alive_d881a351-e7fa-40f2-acba-efe7f8383100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2022 14:36:53] "GET /_alive_d881a351-e7fa-40f2-acba-efe7f8383100 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


### Add apartment list too

In [122]:
apt_list_df = pd.read_csv("https://assets.ctfassets.net/jeox55pd4d8n/2QcueWduZDvVORgKJtXveW/407262141f9b1669748e6e147f112c13/Apartment_List_Rent_Estimates_City_2022_2.csv")
apt_list_df.dropna(inplace=True)

In [124]:
apt_list_df.drop("FIPS_Code", "Population")

,City_Name,FIPS_Code,Population,Bedroom_Size,2017_01,2017_02,2017_03,2017_04,2017_05,2017_06,...,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12,2022_01,2022_02
0,"Birmingham, AL",107000.0,212021.0,_Overall,825.00,829.00,835.00,843.00,852.00,854.00,...,961.00,980.00,1007.00,1019.00,1026.00,1019.00,1014.00,1019.00,1031.00,1041.00
1,"Birmingham, AL",107000.0,212021.0,_Studio,730.00,734.00,739.00,746.00,753.00,756.00,...,850.00,867.00,890.00,902.00,907.00,902.00,897.00,901.00,912.00,921.00
2,"Birmingham, AL",107000.0,212021.0,1br,693.00,697.00,702.00,709.00,716.00,718.00,...,808.00,824.00,846.00,857.00,862.00,857.00,852.00,856.00,867.00,875.00
3,"Birmingham, AL",107000.0,212021.0,2br,816.00,821.00,827.00,834.00,843.00,846.00,...,952.00,970.00,996.00,1009.00,1015.00,1009.00,1004.00,1008.00,1021.00,1031.00
4,"Birmingham, AL",107000.0,212021.0,3br,1066.00,1072.00,1079.00,1089.00,1101.00,1104.00,...,1243.00,1267.00,1301.00,1317.00,1325.00,1317.00,1310.00,1317.00,1332.00,1346.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,"Wauwatosa, WI",5584675.0,47882.0,_Studio,NA,NA,NA,980.00,1004.00,992.00,...,960.00,999.00,1046.00,1058.00,1056.00,1020.00,997.00,1003.00,1015.00,1028.00
3818,"Wauwatosa, WI",5584675.0,47882.0,1br,NA,NA,NA,1140.00,1167.00,1153.00,...,1116.00,1162.00,1216.00,1230.00,1228.00,1186.00,1159.00,1166.00,1180.00,1195.00
3819,"Wauwatosa, WI",5584675.0,47882.0,2br,NA,NA,NA,1222.00,1250.00,1235.00,...,1196.00,1245.00,1303.00,1318.00,1316.00,1271.00,1242.00,1250.00,1265.00,1281.00
3820,"Wauwatosa, WI",5584675.0,47882.0,3br,NA,NA,NA,1495.00,1530.00,1512.00,...,1464.00,1524.00,1595.00,1613.00,1610.00,1556.00,1520.00,1530.00,1548.00,1568.00


In [ ]:
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")

# Apartments

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

import ApartmentsScraperHelper

bb = "xtqrw9_u9Isz8svW"
# pull the same code everytime but for San Marco
ApartmentsScraperHelper.main(bb = bb)

pd.read_csv(bb+".csv")

NameError: name 'csv' is not defined